In [ ]:
# default_exp workflows

# workflows
> Execute workflows on Server

In [ ]:
#hide
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# export
from typing import List
import requests

from yx_motor.api import API
from yx_motor.files import Files
from yx_motor.jobs import Jobs


class Workflows:
    "Class for workflow-related API actions"

    def __init__(self, api: API):
        self.api = api
        self.base_endpoint = "workflows/"
        self.files = Files(self.api)
        self.jobs = Jobs(self.api)

    @staticmethod
    def build_schedule_payload(schedule_name: str, asset_id: str):
        return {"schedule": {"name": schedule_name, "assetId": asset_id,}}

    def get_questions(self, asset_id: str):
        """Return question payload for an asset of type analytic app."""
        response = self.api.post(url=f"workflows/{asset_id}/questions")
        return response

    def get_vfs_inputs(self, asset_id: str):
        """Return the list of vfs inputs for a given workflow asset."""
        response = self.api.get(url=f"workflows/{asset_id}/vfsInputs")
        return response

    def update_vfs_inputs(self, asset_id: str, tool_id: str):
        pass

    def get_workflow_dependencies(self, asset_id: str):
        """Return the list of asset dependencies for a given workflow asset"""
        response = self.api.get(url=f"workflows/{asset_id}/dependencies")
        return response

    def update_workflow_dependencies(self, asset_id: str):
        pass

    def add_workflow_dependency(self, asset_id: str):
        pass

    def add_workflow_dependency_connection(self, asset_id: str, tool_id: str):
        pass

    def run_workflow(
        self, asset_id: str, schedule_name: str, vfs_inputs: List[str] = None
    ):
        """Schedule a workflow to be executed instantaneously.
           Returns a schedule object."""
        response = self.api.post(
            url="workflows/run",
            json=self.build_schedule_payload(schedule_name, asset_id),
        )
        return response

    def download_workflow_results(self, schedule_id: str, download_path: str):
        # TODO: Input validation
        workflow_job = self.jobs.get_job(params={"scheduleId": schedule_id})
        # TODO: Need to check the status of the job, for completion
        # and/or error state before trying to get outputs.

        # TODO: Add validation, error handling here
        output_asset_id = self.get_wf_job_output(workflow_job)[0]

        response = self.files.download_file(
            file_uuid=output_asset_id, download_path=download_path
        )

        return response

    @staticmethod
    def get_wf_job_output(wf_job: object):
        jobs_object = wf_job.json()["jobs"]
        job = jobs_object[0]
        outputs = job["outputs"]
        output_asset_id_list = [file["assetId"] for file in outputs]
        return output_asset_id_list

In [ ]:
#hide

######################### Integration Test Scaffolding ############################
from yx_motor.client import Client

from private import server_vars

base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

dev_client = Client(base_url=base_url,
                    login_email=login_email,
                    login_pwd=login_pwd)

workflows = Workflows(dev_client.api)

#TODO Add to a helpers module

from pathlib import Path

def seed_workflow(dev_client: object, 
                  workflow_path: str, 
                  upload_path: str):
    workflow_path = Path(workflow_path)
    workflow_name = workflow_path.name
    
    workflow_response = dev_client.files.upload_file(filename=workflow_path,
                                                     upload_path=f"{upload_path}{workflow_name}",
                                                     conflict_action="MERGE").json()
    
    return workflow_response


questions_workflow_path = Path(r'test_workflows/simple_questions.yxwz')

workflow_a_path = Path(r'WorkflowA.yxmd')

workflow_b_path = Path(r'WorkflowB.yxmd')

aah_upload_path = r'/Workspaces/Public/'

questions_workflow = seed_workflow(dev_client, 
                                   questions_workflow_path, 
                                   aah_upload_path)

workflow_a_json = seed_workflow(dev_client, 
                                workflow_a_path, 
                                aah_upload_path)

workflow_b_json = seed_workflow(dev_client, 
                                workflow_b_path, 
                                aah_upload_path)

workflow_a_path = workflow_a_json['path']
workflow_a_uuid = workflow_a_json['uuid']
workflow_a_content_id = workflow_a_json['contentId']

workflow_b_path = workflow_b_json['path']
workflow_b_uuid = workflow_b_json['uuid']
workflow_b_content_id = workflow_b_json['contentId']

questions_wf_path = questions_workflow['path']
questions_wf_uuid = questions_workflow['uuid']
questions_wf_content_id = questions_workflow['contentId']

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/authenticate/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Content-Length': '60'}
Response Status: 200
POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'path': '/Workspaces/Public/simple_questions.yxwz', 'conflict_action': 'MERGE', 'Cookie': 'ayxSession=s%3A1113b263-7c49-40e1-a598-b28d9acf96c3.PGPJivlitUSHciX9Sq06%2FAmAe7V%2B74IlUVGTCm3ar%2Bs', 'Content-Length': '22646'}
Response Status: 200
POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,de

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.get_questions)

<h4 id="Workflows.get_questions" class="doc_header"><code>Workflows.get_questions</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.get_questions</code>(**`asset_id`**:`str`)

Return question payload for an asset of type analytic app.

In [ ]:
workflows = Workflows(dev_client.api)

In [ ]:
asset_uid = questions_wf_uuid

In [ ]:
workflows.get_questions(asset_id=asset_uid).json()

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/workflows/473e09c3-9a30-4e84-b5ee-04ed6f4d586b/questions
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Aa9a20c88-e0c5-4639-ba6e-1b05f2c5a067.hsEwETIipdYRtBcRPkjvo8KEXMYVPiwQfetSG9gBq0s', 'Content-Length': '0'}
Response Status: 200


{'fieldMap': {},
 'questions': {'Questions': {'Question': [{'Type': 'Tab',
     'Description': 'Questions (tab)',
     'Name': 'Tab (1)',
     'ToolId': {'@value': '1'},
     'Questions': {'Question': [{'Type': 'TextBox',
        'Description': 'Company Name (textbox)',
        'Name': 'Text Box (2)',
        'ToolId': {'@value': '2'},
        'Default': 'Alteryx',
        'Password': {'@value': 'False'},
        'Multiline': {'@value': 'False'},
        'Hidden': {'@value': 'False'}},
       {'Type': 'NumericUpDown',
        'Description': 'Employees (numeric up/down)',
        'Name': 'Numeric Up Down (8)',
        'ToolId': {'@value': '8'},
        'Minimum': {'@value': '0'},
        'Maximum': {'@value': '1000'},
        'Increment': {'@value': '1'},
        'Default': {'@value': '100'},
        'Decimals': {'@value': '0'}},
       {'Type': 'Label',
        'Description': 'Select a radio button below and then a value from the listbox or dropdown (label)',
        'Name': 'Label (12

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.get_vfs_inputs)

<h4 id="Workflows.get_vfs_inputs" class="doc_header"><code>Workflows.get_vfs_inputs</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.get_vfs_inputs</code>(**`asset_id`**:`str`)

Return the list of vfs inputs for a given workflow asset.

In [ ]:
workflows = Workflows(dev_client.api)
workflows.get_vfs_inputs(asset_id=asset_uid).json()

GET sent to: https://test.aep-mono.devops.alteryx.com/api/v1/workflows/473e09c3-9a30-4e84-b5ee-04ed6f4d586b/vfsInputs
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Aa9a20c88-e0c5-4639-ba6e-1b05f2c5a067.hsEwETIipdYRtBcRPkjvo8KEXMYVPiwQfetSG9gBq0s'}
Response Status: 200


[]

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.get_workflow_dependencies)

<h4 id="Workflows.get_workflow_dependencies" class="doc_header"><code>Workflows.get_workflow_dependencies</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.get_workflow_dependencies</code>(**`asset_id`**:`str`)

Return the list of asset dependencies for a given workflow asset

In [ ]:
workflows = Workflows(dev_client.api)
workflows.get_workflow_dependencies(asset_id=asset_uid).json()

GET sent to: https://test.aep-mono.devops.alteryx.com/api/v1/workflows/473e09c3-9a30-4e84-b5ee-04ed6f4d586b/dependencies
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Aa9a20c88-e0c5-4639-ba6e-1b05f2c5a067.hsEwETIipdYRtBcRPkjvo8KEXMYVPiwQfetSG9gBq0s'}
Response Status: 200


[]

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.run_workflow)

<h4 id="Workflows.run_workflow" class="doc_header"><code>Workflows.run_workflow</code><a href="__main__.py#L50" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.run_workflow</code>(**`asset_id`**:`str`, **`schedule_name`**:`str`, **`vfs_inputs`**:`List`\[`str`\]=*`None`*)

Schedule a workflow to be executed instantaneously.
Returns a schedule object.

**Arguments**:

- asset_id: Unique VFS identifier for the workflow you want to execute.
- schedule_name: Optional: Name of the schedule that will be created when workflow execution is triggered.
- vfs_inputs: TODO, need to research this more and test it.

In [ ]:
asset_uid = questions_wf_uuid

In [ ]:
workflows = Workflows(dev_client.api)
workflows.run_workflow(asset_id=asset_uid,
                       schedule_name="jp_client_test").json()

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/workflows/run
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A00c3bae2-9735-4fdb-8ae4-4b57f27c738f.ZGV%2Fvd7nTS%2FwwrK1g5QmdthQ5vHpi8JV6vxiQV%2BohCE', 'Content-Length': '91'}
Response Status: 200


{'scheduleId': '76ea8585-839d-444c-9f99-7c2abb526932',
 'name': 'jp_client_test',
 'userId': '352d4b64-1477-4f46-a9fe-b29031639783',
 'assetId': '275bfa23-f718-4a19-a587-f45c33bebaba',
 'assetVersion': None,
 'runAsId': None,
 'toolConnections': None,
 'status': 'active',
 'startDate': None,
 'endDate': None,
 'lastRun': None,
 'nextRun': None,
 'frequencyInterval': 'manual',
 'frequencyRule': None,
 'type': 'immediate',
 'isDisabled': False,
 'isDeleted': False,
 'comment': None,
 'siteId': '9964463b-7c64-42c8-856f-38b93bd14c42',
 'parameters': {'runMode': 0},
 'analyticAppAnswers': None,
 'inputFiles': None,
 'creationDate': '2020-05-05T20:15:06.436Z',
 'lastUpdate': '2020-05-05T20:15:06.436Z',
 'outputLocation': None,
 'timezone': 'UTC',
 'utcOffset': 0,
 'priority': 50}

In [ ]:
from nbdev.showdoc import *
show_doc(Workflows.download_workflow_results)

<h4 id="Workflows.download_workflow_results" class="doc_header"><code>Workflows.download_workflow_results</code><a href="__main__.py#L60" class="source_link" style="float:right">[source]</a></h4>

> <code>Workflows.download_workflow_results</code>(**`schedule_id`**:`str`, **`download_path`**:`str`)



In [ ]:
workflows = Workflows(dev_client.api)
response = workflows.run_workflow(asset_id=asset_uid,
                       schedule_name="jp_client_test").json()

POST sent to: https://test.aep-mono.devops.alteryx.com/api/v1/workflows/run
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A1113b263-7c49-40e1-a598-b28d9acf96c3.PGPJivlitUSHciX9Sq06%2FAmAe7V%2B74IlUVGTCm3ar%2Bs', 'Content-Length': '91'}
Response Status: 200


In [ ]:
schedule_id = response['scheduleId']

In [ ]:
workflows.download_workflow_results(schedule_id=schedule_id, 
                                    download_path='wf_out_test.csv')

GET sent to: https://test.aep-mono.devops.alteryx.com/api/v1/jobs/?scheduleId=4f140ceb-fdba-46d3-b78c-01430134e391
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A1113b263-7c49-40e1-a598-b28d9acf96c3.PGPJivlitUSHciX9Sq06%2FAmAe7V%2B74IlUVGTCm3ar%2Bs'}
Response Status: 200
GET sent to: https://test.aep-mono.devops.alteryx.com/api/v1/files/content?id=df2b5d7f-7cfc-4460-bc63-209846310a78
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A1113b263-7c49-40e1-a598-b28d9acf96c3.PGPJivlitUSHciX9Sq06%2FAmAe7V%2B74IlUVGTCm3ar%2Bs'}
Response Status: 200


<Response [200]>

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 01_client.ipynb.
Converted 02_jobs.ipynb.
Converted 03_api.ipynb.
Converted 04_authenticate.ipynb.
Converted 05_files.ipynb.
Converted 06_workflows.ipynb.
Converted index.ipynb.


In [ ]:
from nbdev.sync import script2notebook
script2notebook()